<a href="https://colab.research.google.com/github/po04227/Github_sw/blob/master/MLP_MNIST_Tensorflow_Early_Stopping_DropOut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import numpy as np
from IPython.display import Image

In [ ]:
# MLP Architecture
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/dropout.png", width=500, height=250)

In [ ]:
# Collect MNIST Data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [ ]:
# Split train data into train and validation data
x_val  = x_train[50000:60000]
x_train = x_train[0:50000]
y_val  = y_train[50000:60000]
y_train = y_train[0:50000]

In [ ]:
print("train data has " + str(x_train.shape[0]) + " samples")
print("every train data is " + str(x_train.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

train data has 50000 samples
every train data is 28 * 28 image


In [ ]:
print("validation data has " + str(x_val.shape[0]) + " samples")
print("every train data is " + str(x_val.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

validation data has 10000 samples
every train data is 28 * 28 image


In [ ]:
# sample to show gray scale values
print(x_train[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


In [ ]:
# sample to show labels for first train data to 10th train data
print(y_train[0:9])

[5 0 4 1 9 2 1 3 1]


In [ ]:
print("test data has " + str(x_test.shape[0]) + " samples")
print("every test data is " + str(x_test.shape[1]) 
      + " * " + str(x_test.shape[2]) + " image")

test data has 10000 samples
every test data is 28 * 28 image


In [ ]:
# Reshape
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/reshape_mnist.png", width=500, height=250)

In [ ]:
x_train = x_train.reshape(50000, 784)
x_val = x_val.reshape(10000, 784)
x_test = x_test.reshape(10000, 784)

print(x_train.shape)
print(x_test.shape)

(50000, 784)
(10000, 784)


In [ ]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [ ]:
# Normalize data
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

In [ ]:
# label to one hot encoding value
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [ ]:
# Tensorflow MLP Graph
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_mlp_mnist.png", width=500, height=250)

In [ ]:
# Drop Out
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

In [ ]:
def mlp(x):
    # hidden layer1
    w1 = tf.Variable(tf.random_uniform([784,256]))
    b1 = tf.Variable(tf.zeros([256]))
    h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    # hidden layer2
    w2 = tf.Variable(tf.random_uniform([256,128]))
    b2 = tf.Variable(tf.zeros([128]))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    h2_drop = tf.nn.dropout(h2, keep_prob)
    # output layer
    w3 = tf.Variable(tf.random_uniform([128,10]))
    b3 = tf.Variable(tf.zeros([10]))
    logits= tf.matmul(h2_drop, w3) + b3
    
    return logits

In [ ]:
logits = mlp(x)

In [ ]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=y))

In [ ]:
train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)

In [ ]:
# Early Stopping
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/early_stop.png", width=500, height=250)

In [ ]:
# initialize
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# train hyperparameters
epoch_cnt = 300
batch_size = 1000
iteration = len(x_train) // batch_size

earlystop_threshold = 5
earlystop_cnt = 0

In [ ]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    prev_train_acc = 0.0
    max_val_acc = 0.0
    
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            _, loss = sess.run([train_op, loss_op], 
                               feed_dict={x: x_train[start: end], y: y_train[start: end], 
                                          keep_prob: 0.9})
            start += batch_size; end += batch_size
            # Compute train average loss
            avg_loss += loss / iteration
            
        # Validate model
        preds = tf.nn.softmax(logits)  # Apply softmax to logits
        correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        # train accuracy
        cur_train_acc = accuracy.eval({x: x_train, y: y_train,keep_prob: 1.0})
        # validation accuarcy
        cur_val_acc = accuracy.eval({x: x_val, y: y_val, keep_prob: 1.0})
        # validation loss
        cur_val_loss = loss_op.eval({x: x_val, y: y_val,keep_prob: 1.0})
        
        print("epoch: "+str(epoch)+
              ", train acc: " + str(cur_train_acc) +
              ", val acc: " + str(cur_val_acc) )
              #', train loss: '+str(avg_loss)+
              #', val loss: '+str(cur_val_loss))
        
        if cur_val_acc < max_val_acc:
            if cur_train_acc > prev_train_acc or cur_train_acc > 0.99:
                if earlystop_cnt == earlystop_threshold:
                    print("early stopped on "+str(epoch))
                    break
                else:
                    print("overfitting warning: "+str(earlystop_cnt))
                    earlystop_cnt += 1
            else:
                earlystop_cnt = 0
        else:
            earlystop_cnt = 0
            max_val_acc = cur_val_acc
            # Save the variables to file.
            save_path = saver.save(sess, "model/model.ckpt")
        prev_train_acc = cur_train_acc

epoch: 0, train acc: 0.25734, val acc: 0.2697
epoch: 1, train acc: 0.41154, val acc: 0.4301
epoch: 2, train acc: 0.52338, val acc: 0.5408
epoch: 3, train acc: 0.60968, val acc: 0.6319
epoch: 4, train acc: 0.6658, val acc: 0.6819
epoch: 5, train acc: 0.70472, val acc: 0.7156
epoch: 6, train acc: 0.7313, val acc: 0.7445
epoch: 7, train acc: 0.7567, val acc: 0.767
epoch: 8, train acc: 0.77766, val acc: 0.7886
epoch: 9, train acc: 0.79752, val acc: 0.8039
epoch: 10, train acc: 0.81356, val acc: 0.8208
epoch: 11, train acc: 0.82868, val acc: 0.8335
epoch: 12, train acc: 0.83904, val acc: 0.8425
epoch: 13, train acc: 0.85026, val acc: 0.8522
epoch: 14, train acc: 0.8594, val acc: 0.86
epoch: 15, train acc: 0.86636, val acc: 0.8678
epoch: 16, train acc: 0.87266, val acc: 0.8735
epoch: 17, train acc: 0.87858, val acc: 0.8781
epoch: 18, train acc: 0.88306, val acc: 0.8839
epoch: 19, train acc: 0.8884, val acc: 0.8869
epoch: 20, train acc: 0.89288, val acc: 0.8926
epoch: 21, train acc: 0.8962, v

In [ ]:
# Testing with the best epoch
# Start testing
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "model/model.ckpt")
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("[Test Accuracy] :", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model/model.ckpt
[Test Accuracy] : 0.957
